<a href="https://colab.research.google.com/github/kundajelab/feature_interactions/blob/master/ComputeModelPerfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/kundajelab/feature_interactions/raw/6f96084/av/data/test_simulation.simdata.gz 
!wget https://github.com/kundajelab/feature_interactions/raw/6f96084/av/data/test_neg_labels.txt.gz
!wget https://github.com/kundajelab/feature_interactions/raw/6f96084/av/data/test_pos_labels.txt.gz

In [0]:
!gunzip -f *.gz

In [0]:
!wget https://github.com/kundajelab/feature_interactions/raw/77d29d1/av/trained_models.tgz

In [0]:
!tar -xzf trained_models.tgz

In [0]:
!ls trained_models/

In [0]:
!pip install simdna

In [0]:
import simdna
from simdna import synthetic

test_data = synthetic.read_simdata_file("test_simulation.simdata")

In [0]:
import numpy as np


#this is set up for 1d convolutions where examples
#have dimensions (len, num_channels) 
#the channel axis is the axis for one-hot encoding.
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return


def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1


def anscombe_transform(vals):
  return 2*np.sqrt(vals + 3.0/8)


def inverse_anscombe_transform(vals):
  return np.square(vals/2.0) - 3.0/8


def read_labels_and_oracle(filename):
  labels = anscombe_transform(np.array([float(x.split("\t")[0]) for
                                          x in open(filename)]))
  oracle = anscombe_transform(np.array([float(x.split("\t")[1]) for
                                          x in open(filename)]))
  return labels, oracle


test_onehot_data = np.array([one_hot_encode_along_channel_axis(seq)
                              for seq in test_data.sequences])

test_pos_labels, test_pos_oracle =\
  read_labels_and_oracle("test_pos_labels.txt")
test_neg_labels, test_neg_oracle =\
  read_labels_and_oracle("test_neg_labels.txt")

In [0]:
import glob

model_files = glob.glob("trained_models/*.h5")

In [0]:
%tensorflow_version 1.x

In [0]:
import keras
from collections import OrderedDict
import scipy.stats

def get_modelname_to_loss(modelfiles, labels):
  modelname_to_loss = OrderedDict()
  modelname_to_corr = OrderedDict()
  for model_file in modelfiles:
    print("On",model_file)
    model = keras.models.load_model(model_file)
    preds = np.squeeze(model.predict(test_onehot_data))
    loss = np.mean(np.square(preds-labels))
    corr = scipy.stats.pearsonr(preds, labels)[0]
    modelname_to_loss[model_file.split("/")[-1]] = loss
    modelname_to_corr[model_file.split("/")[-1]] = corr
  return modelname_to_loss, modelname_to_corr

poscontrol_modelname_to_loss, poscontrol_modelname_to_corr = get_modelname_to_loss(
    modelfiles=glob.glob("trained_models/poscontrol_*.h5"),
    labels=test_pos_labels)
negcontrol_modelname_to_loss, negcontrol_modelname_to_corr = get_modelname_to_loss(
    modelfiles=glob.glob("trained_models/negcontrol_*.h5"),
    labels=test_neg_labels)


In [0]:
import json

def pp_json_dump(obj):
  return json.dumps(obj, sort_keys=True, indent=4, separators=(',', ': '))

with open("poscontrol_modelname_to_loss.json",'w') as fh:
  fh.write(pp_json_dump(poscontrol_modelname_to_loss))

with open("negcontrol_modelname_to_loss.json",'w') as fh:
  fh.write(pp_json_dump(negcontrol_modelname_to_loss))

with open("poscontrol_modelname_to_corr.json",'w') as fh:
  fh.write(pp_json_dump(poscontrol_modelname_to_corr))

with open("negcontrol_modelname_to_corr.json",'w') as fh:
  fh.write(pp_json_dump(negcontrol_modelname_to_corr))

In [0]:
from google.colab import files

files.download("poscontrol_modelname_to_loss.json")
files.download("negcontrol_modelname_to_loss.json")

files.download("poscontrol_modelname_to_corr.json")
files.download("negcontrol_modelname_to_corr.json")